# Example usage of REST API functions
Below are some example usage functions for showCnt(content) and getValues that are available RESTful GET calls
from worldofcode.org. 
**showCnt** displays related information about the sha passed to it, such as the files contained in a tree sha, the content of a blob sha, and the tree, parent commit, and author of a commit sha. 
**getValues** returns different mapping relationships of a given sha, such as the files associated with a commit (c2f), or the projects associated with a blob (b2P). 

*Please note: these functions only support SHA1 hashes. Functionality for other objects, such as authors, file names, and project URIs have not yet been implemented but will be soon!* 

Utilizing the `requests` and `json` libraries, we are able to make these GET calls from any server, local or remote, through a Jupyter Notebook

In [45]:
import requests, json
import re

def showCnt (type, sha1):
  url='http://worldofcode.org/api/lookup?command=showCnt' + '&type='+ type + '&sha1=' + sha1
  r = requests.get(url)
  res = json.loads(r.content)['stdout']
  if type == 'commit':
    res = res.split(';')
    print('Tree:'+res[1]+'\nParent:'+res[2]+'\nAuthor:'+res[3]+'\n')

  elif type == 'blob':
    if not res:
        print("Blob content can't be displayed")
    else:
        print(res)

  elif type == 'tree':
    res = re.split('\n|;', res)
    print("No. of files in this tree: " + str(len(res[1:])/3))
    for i in range(0,len(res)-3,3):
        print('Mode:'+res[i]+'  Blob sha:'+res[i+1]+'  Filename:'+res[i+2])
    
  else:
    print("Please specify either tree, commit, or blob.")


def getValues (map_type, sha1):
  url='http://worldofcode.org/api/lookup?command=getValues' + '&type='+ map_type + '&sha1=' + sha1
  r = requests.get(url)
  return (json.loads(r.content)['stdout']).split(';')[1:]

Using these 2 predefined fuctions, different calls can be made to display various information:

In [43]:
showCnt("commit", "009d7b6da9c4419fe96ffd1fffb2ee61fa61532a")

Tree:464ac950171f673d1e45e2134ac9a52eca422132
Parent:dddff9a89ddd7098a1625cafd3c9d1aa87474cc7
Author:Warner Losh <imp@FreeBSD.org>



In [40]:
showCnt("blob", 'a8fe822f075fa3d159a203adfa40c3f59d6dd999')

# $FreeBSD$
#	@(#)COPYRIGHT	8.2 (Berkeley) 3/21/94

The compilation of software known as FreeBSD is distributed under the
following terms:

Copyright (C) 1992-2004 The FreeBSD Project. All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions
are met:
1. Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY AUTHOR AND CONTRIBUTORS ``AS IS'' AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
ARE DISCLAIMED.  IN NO EVENT SHALL AUTHOR OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, 

In [39]:
showCnt("tree", '464ac950171f673d1e45e2134ac9a52eca422132')

No. of files in this tree: 31
Mode:100644  Blob sha:a8fe822f075fa3d159a203adfa40c3f59d6dd999  Filename:COPYRIGHT
Mode:100644  Blob sha:fd302b41386a29c3d19110a394b3f377d8c8a55a  Filename:Caldera-license.pdf
Mode:100644  Blob sha:ea1a840100ad18f66056e081616c643455d25316  Filename:LICENSE
Mode:100644  Blob sha:80ef82af7457a1a003c279afa4044598ddcedb8c  Filename:MAINTAINERS
Mode:100644  Blob sha:51bd582efda09dab0dc1cab14f08f21503e83f45  Filename:Makefile
Mode:100644  Blob sha:381a0f6f3f0f642bb62f5cf8e6d62806debba71e  Filename:Makefile.inc1
Mode:100644  Blob sha:70402b9973a93910b0f3d59bcb1c8e7e8209112e  Filename:README
Mode:100644  Blob sha:7b179b8659417b13b0a3d5c5f184ed29f907b67d  Filename:README.md
Mode:100644  Blob sha:9f896c36b9511942bc13da08b44680b7e47c7bf4  Filename:UPDATING
Mode:100644  Blob sha:6b04d250c1d465e1ee8831c30f58ac22b7f8800d  Filename:UPDATING.64BTT
Mode:040000  Blob sha:def5f8c3e89849bc0a872eb9b7f995c65f242595  Filename:bin
Mode:040000  Blob sha:86ffdacb2fe6d9d7d696899cded

The list of available mappings is as follows: 
```
* a2b 	* a2c 	* a2f	* a2ft		
* a2p 	* a2trp0 
* b2a	* b2c 	* b2f 	* b2ob	* b2tk
* c2b 	* c2cc	* c2f 	* c2h	* c2pc
* c2p 	* c2P	* c2ta 	* c2td
* f2a	* f2b 	* f2c 		
* p2a 	* p2c 	* P2c
* td2c	* td2f
```
The items that the letters reference: 
```
a = Author, b = Blob, c = Commit, cc = Child Commit
f = File, h = Head Commit, ob = Parent Blob, p = Project
pc = Parent Commit, P = Forked/Root Project (see Note below)
ta = Time Author, td = Tdiff
tk = Tokens (ctags), trp = Torvalds Path
```
Note: c2P returns the most central repository for this commit, and does not include repos that forked off of this commit. 
      P2c returns ALL commits associated with this repo, including commits made to forks of this particular repo.

Some examples are listed below:

In [46]:
getValue('c2b', 'd7cad81093cd805110291f84d64d385557d0ffba')

[u'9808d70aed03290c648b983ea404446779eff501\n']

In [47]:
getValue('c2f', 'd7cad81093cd805110291f84d64d385557d0ffba')

[u'lib/lz4.c\n']

Just as showCnt's output has been formatted for easier viewing, getValues can also be formatted. As opposed to viewing a list of items in an array,

In [48]:
getValue('b2P', '9808d70aed03290c648b983ea404446779eff501')

[u'0-wiz-0_libuv',
 u'06094051_librdkafka',
 u'1010101012101_borg',
 u'3370sohail_gecko-dev',
 u'540KJ_root',
 u'6180_python-lz4',
 u'Alexhuszagh_c-blosc',
 u'Bambooie_gdsfmt',
 u'cactus74_fst',
 u'darkskygit_simple_kv',
 u'draede_cx',
 u'dudw_libportable',
 u'fangq_matzip',
 u'girdharshubham_gokafka',
 u'git.bioconductor.org_packages_gdsfmt',
 u'jmoiron_golz4',
 u'karubabu_quickbms',
 u'liliilli_Dy',
 u'lmtwga_lzbench',
 u'marcelorl_graphql-real-time-example',
 u'pharaoh1_7-Zip-zstd',
 u'scalarwaves_zbox',
 u'silnrsi_grcompiler',
 u'tafia_fstlib',
 u'ushiiwaka_ps2-packer',
 u'windreamer_py-lz4framed\n']

the output can be formatted as so:

In [52]:
v = getValue('b2P', '9808d70aed03290c648b983ea404446779eff501')
for p in v: print (p+'\n')

0-wiz-0_libuv

06094051_librdkafka

1010101012101_borg

3370sohail_gecko-dev

540KJ_root

6180_python-lz4

Alexhuszagh_c-blosc

Bambooie_gdsfmt

cactus74_fst

darkskygit_simple_kv

draede_cx

dudw_libportable

fangq_matzip

girdharshubham_gokafka

git.bioconductor.org_packages_gdsfmt

jmoiron_golz4

karubabu_quickbms

liliilli_Dy

lmtwga_lzbench

marcelorl_graphql-real-time-example

pharaoh1_7-Zip-zstd

scalarwaves_zbox

silnrsi_grcompiler

tafia_fstlib

ushiiwaka_ps2-packer

windreamer_py-lz4framed


